In [1]:
from py4j.java_gateway import JavaGateway

In [2]:
gateway = JavaGateway()

In [6]:
sarray = gateway.new_array(gateway.jvm.java.lang.String,2)
sarray[0] = '/home/hadi/hLDA_GitHub/ISI_Abstract_train.txt'
#sarray[1] = '/home/hadi/hLDA_GitHub/ISI_Abstract_test.txt'

In [7]:
pipeList= gateway.jvm.java.util.ArrayList();
pipeList.add(gateway.jvm.cc.mallet.pipe.CharSequenceLowercase());
pipeList.add( gateway.jvm.cc.mallet.pipe.CharSequence2TokenSequence\
             (gateway.jvm.java.util.regex.Pattern.compile("\\p{L}[\\p{L}-_\\p{N}]+[\\p{L}\\p{N}]")));
pipeList.add(gateway.jvm.cc.mallet.pipe.TokenSequenceRemoveStopwords(gateway.jvm.java.io.File\
                                                    ("stoplists/fr.txt"), "UTF-8", False, False, False));
pipeList.add(gateway.jvm.cc.mallet.pipe.TokenSequenceRemoveStopwords(gateway.jvm.java.io.File\
                                                    ("stoplists/en.txt"), "UTF-8", False, False, False));
pipeList.add(gateway.jvm.cc.mallet.pipe.TokenSequence2FeatureSequence());

In [8]:
instances = gateway.jvm.cc.mallet.types.InstanceList(gateway.jvm.cc.mallet.pipe.SerialPipes(pipeList))

#testInstances = gateway.jvm.cc.mallet.types.InstanceList(instances.getPipe());

insfileReader = gateway.jvm.java.io.InputStreamReader(gateway.jvm.java.io.FileInputStream \
                                            (gateway.jvm.java.io.File(sarray[0])), "UTF-8");

#testfileReader = gateway.jvm.java.io.InputStreamReader(gateway.jvm.java.io.FileInputStream \
#                                           (gateway.jvm.java.io.File(sarray[1])), "UTF-8");

instances.addThruPipe(gateway.jvm.cc.mallet.pipe.iterator.CsvIterator \
    (insfileReader, gateway.jvm.java.util.regex.Pattern.compile("^(\\S*)[\\s,]*(\\S*)[\\s,]*(.*)$"), 3, 2, 1)); 

#testInstances.addThruPipe(gateway.jvm.cc.mallet.pipe.iterator.CsvIterator \
#    (testfileReader, gateway.jvm.java.util.regex.Pattern.compile("^(\\S*)[\\s,]*(\\S*)[\\s,]*(.*)$"),3, 2, 1)); 

## LDA  implementation

In [9]:
Numtopics = 20
s0 = gateway.jvm.cc.mallet.topics.ParallelTopicModel(Numtopics)
s0.addInstances(instances)
s0.setOptimizeInterval(10)
s0.estimate()

In [10]:
#inferencer = s0.getInferencer()

#testOut = gateway.jvm.java.io.PrintWriter(gateway.jvm.java.io.FileWriter\
#                            ("/home/hadi/hLDA_GitHub/topicdistribution.csv"));
#inferencer. writeInferredDistributions(testInstances, testOut, 1000, 1, 0, 0.0, -1);
#testOut.close();


## hLDA implementation 

In [13]:
s1= gateway.jvm.cc.mallet.topics.HierarchicalLDA()        

# set parameter
s1.setAlpha(10.0)
s1.setGamma(1.0);
s1.setEta(0.1);

# set level here, the default is 3
tmpRand=gateway.jvm.cc.mallet.util.Randoms()
s1.initialize(instances, instances, 3,  tmpRand);
s1.estimate(200)